In [1]:
import librosa
import os
import IPython.display as lpd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
import pandas as pd


#get list of all files with labels
data_path = './asset/birdclef-2021/train_short_audio/'

df = pd.read_csv('./asset/birdclef-2021/train_metadata.csv')
blist=df['primary_label'].unique()

num_count = 1
call_list=list()
for sec,pri,filename in zip(df['secondary_labels'],df['primary_label'],df['filename']):
    sec_2 = list(sec.replace("'",'').replace('[','').replace(']','').split(','))
    sec_2.append(pri)
    if sec_2[0]=='':
        sec_2=sec_2[1:]
    call_list.append([os.path.join(data_path,pri,filename),sec_2])
len(call_list)

blist=list(blist)
blist.append('nocall')


In [2]:
def get_call_window(wav,duration=32000*5,mode='precise'):
    #Todo : implementing get multiple windows option would be needed
    
    if not mode in ['precise','fast']:
        raise ValueError('get_call_window mode parameter allow "precise" and "fast"')
    
    mean = sum(np.abs(wav))/len(wav)
    call = None
    silence = None
    for i in range(0,len(wav),32000):
        if i+duration > len(wav):
            break
        if sum(np.abs(wav[i:i+duration]))/duration > mean:
            if call is None: call = wav[i:i+duration]
            else:
                if sum(np.abs(call))/duration < sum(np.abs(wav[i:i+duration]))/duration:
                    call = wav[i:i+duration]
        else:
            if silence is None: silence = wav[i:i+duration]
            else:
                if sum(np.abs(silence))/duration > sum(np.abs(wav[i:i+duration]))/duration:
                    silence = wav[i:i+duration]
        if mode == 'fast' and (call is not None and silence is not None):
            return call, silence
    return call,silence

def convert_to_mel(wav):
    SPEC_HEIGHT = 64
    SPEC_WIDTH = 256
    NUM_MELS = SPEC_HEIGHT
    HOP_LENGTH = int(32000 * 5 / (SPEC_WIDTH - 1)) # sample rate * duration / spec width - 1 == 627
    FMIN = 500
    FMAX = 12500
    mel_spec = librosa.feature.melspectrogram(y=wav, 
                                              sr=32000, 
                                              n_fft=1024, 
                                              hop_length=HOP_LENGTH, 
                                              n_mels=NUM_MELS, 
                                              fmin=FMIN, 
                                              fmax=FMAX)
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec -= mel_spec.min()
    mel_spec /= mel_spec.max()
    return mel_spec

In [3]:

import copy
import random

pickle_list = copy.deepcopy(call_list)
random.shuffle(pickle_list)

num_data = 400

nocall_bag = list()
nocall_max = num_data//10
new_list = list()
for i,data in enumerate(pickle_list):
    print(i)
    if i > num_data:
        break
    wav, _ =librosa.load(data[0],sr=32000)
    wav = librosa.util.normalize(wav)
    call, silence = get_call_window(wav,mode='fast')
    new_list.append([call,data[1]])
    if nocall_max>len(nocall_bag):
        nocall_bag.append([silence,['nocall']])

new_list = new_list+nocall_bag


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
mel_list = list()
for i,data in enumerate(new_list):
    if i%100==0:
        print(i)
    BCencoding = np.zeros((len(blist)))
    for bird in data[1]:
        if bird in blist:
            BCencoding[blist.index(bird)] = 1
    if data[0] is None:
        continue
    mel = convert_to_mel(data[0])
    mel_list.append([mel,BCencoding])

0
100
200
300
400


In [5]:
import pickle
'''
with open('./asset/temp.pkl','wb') as f:
    pickle.dump(mel_list,f)
''' 
with open('./asset/temp.pkl','rb') as f:
    mel_list = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './asset/temp.pkl'

In [ ]:
import torch 
from modules.ResNet import resnet152 , resnet50
import torch.optim as optim
from IPython.display import display
from torch.cuda.amp import GradScaler, autocast
from utils.loss import AsymmetricLoss
from utils.ema import ModelEma
from torch.optim import lr_scheduler


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#model = resnet152(num_classes = len(blist))
model = resnet152(num_classes = len(blist))
model.to(device)
model.train()

class Averager(object):
    """Compute average for torch.Tensor, used for loss average."""

    def __init__(self):
        self.reset()

    def add(self, v):
        count = v.data.numel()
        v = v.data.sum()
        self.n_count += count
        self.sum += v

    def reset(self):
        self.n_count = 0
        self.sum = 0

    def val(self):
        res = 0
        if self.n_count != 0:
            res = self.sum / float(self.n_count)
        return res
    
def get_multi_batch(dataset,cur,batch_size):
    call = np.concatenate([data[0][np.newaxis,np.newaxis,:,:] for data in dataset[cur:cur+batch_size]])
    lbl = np.concatenate([np.reshape(data[1],(1,len(blist))) for data in dataset[cur:cur+batch_size]])
    call = torch.from_numpy(call).to(device)
    lbl = torch.from_numpy(lbl).to(device)
    
    return call,lbl

dataset = mel_list

ema = ModelEma(model, 0.9997)
loss_avg = Averager()
criterion = AsymmetricLoss(gamma_neg=4, gamma_pos=0, clip=0.05, disable_torch_grad_focal_loss=True)

filtered_parameters = []
params_num = []

scaler = GradScaler()

for p in filter(lambda p: p.requires_grad, model.parameters()):
    filtered_parameters.append(p)
    params_num.append(np.prod(p.size()))
print('Trainable params num : ', sum(params_num))

learning_rate=0.0003
optimizer = optim.Adam(filtered_parameters, lr=learning_rate, betas=(0.9, 0.999))
batch_size = 10
epochs = 100
df = display('go',display_id=True)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(dataset), epochs=epochs,
                                        pct_start=0.2)
for epoch in range(epochs):
    for batch in range(len(dataset)//batch_size):
        
        call, lbl = get_multi_batch(dataset,batch,batch_size)
        #test_data = np.concatenate([data[0][np.newaxis,np.newaxis,:,:] for data in dataset[:batch_size]])
        with autocast():  # mixed precision
            output = model(call).float()  # sigmoid will be done in loss !
        
        loss = criterion(output, lbl)
        loss_avg.add(loss)
        model.zero_grad()

        scaler.scale(loss).backward()
        # loss.backward()

        scaler.step(optimizer)
        scaler.update()
        # optimizer.step()

        scheduler.step()

        ema.update(model)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),5)  # gradient clipping with 5 (Default)
        #https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-6/05-gradient-clipping
        df.update(f'epoch : {epoch} / batch : {batch} / loss : {loss_avg.val().item()}')
        
        del call, lbl, output, loss

In [ ]:
ind2bird = dict()
for i,bird in enumerate(blist):
    ind2bird[i] = bird

In [ ]:
scape_df.keys()

In [ ]:
import torch 
import librosa
import numpy as np
from modules.ResNet import resnet152, resnet50
import torch.optim as optim
from IPython.display import display
from torch.cuda.amp import GradScaler, autocast
from utils.loss import AsymmetricLoss
from utils.ema import ModelEma
from torch.optim import lr_scheduler
import torch.nn.functional as F
from data.preprocess import get_feature_extractor,mono_to_color
import pickle

with open('./train_data/temp_2-003.pkl','rb') as f:
    data = pickle.load(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


scape_df = pd.read_csv('./asset/birdclef-2021/train_soundscape_labels.csv')
scape_path = './asset/birdclef-2021/train_soundscapes/'
scape_list = os.listdir(scape_path)
f1_score_sum = 0
f1_score_num = 0


model = resnet152(num_classes = 397)
#model = resnet50(num_classes = 397)
feature_extractor = get_feature_extractor()


saved_model = './result/model_1620811543.8092575.pth'
#saved_model = './result/lr_0.003_res50.pth'
model.load_state_dict(torch.load(saved_model, map_location=device))
model.to(device)

model.eval()
classes_list = np.array(blist)[:-1]

scape_dic=dict()
for row,sec,birds in zip(scape_df['row_id'],scape_df['seconds'],scape_df['birds']):
    temp = row.split('_')[:-1]
    filename = '_'.join(temp)
    if filename in scape_dic.keys():
        scape_dic[filename].append([sec,birds])
    else:
        scape_dic[filename]=[[sec,birds]]

        

for scape in scape_list:
    if scape.split('.')[-1]!='ogg':
        continue
        
    wav, _ =librosa.load(os.path.join(scape_path,scape),sr=32000)
    wav = librosa.util.normalize(wav)
    
    temp = scape.split('_')[:-1]
    filename = '_'.join(temp)
    for frame in scape_dic[filename]:
        point = frame[0]*32000
        clip = wav[point-32000*5:point]
        #lpd.display(lpd.Audio(clip,rate=32000))
        clip = torch.from_numpy(clip[np.newaxis,:]).to(device)
        
        mel_spec = feature_extractor(clip)
        mel_spec = torch.from_numpy(mono_to_color(mel_spec.cpu().detach().numpy())).to(device)

        #print(mel_spec)
        output = torch.squeeze(F.softmax(model(mel_spec.to(torch.float32))))
        np_output = output.cpu().detach().numpy()
        print(classes_list[np_output>0.3],frame[1].split(' '))
'''

for row in data:
    mel_spec = torch.from_numpy(row[0][np.newaxis,np.newaxis,:,:]).to(device)
    output = torch.squeeze(F.softmax(model(mel_spec.to(torch.float32))))
    np_output = output.cpu().detach().numpy()
    print(classes_list[np_output>0.3],classes_list[row[1]>0.5])
    
'''

In [ ]:
import torch 
import librosa
import numpy as np
from modules.ResNet import resnet152, resnet50
import torch.optim as optim
from IPython.display import display
from torch.cuda.amp import GradScaler, autocast
from utils.loss import AsymmetricLoss
from utils.ema import ModelEma
from torch.optim import lr_scheduler
import torch.nn.functional as F
from data.preprocess import get_feature_extractor,mono_to_color


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#model = resnet152(num_classes = 397)
model = resnet50(num_classes = 397)
feature_extractor = get_feature_extractor()



#saved_model = './result/1620829476.648939.pth'
saved_model = './result/lr_0.003_res50.pth'
model.load_state_dict(torch.load(saved_model, map_location=device))
model.to(device)


data_path = './asset/birdclef-2021/train_soundscapes/10534_SSW_20170429.ogg'
wav, _ =librosa.load(data_path,sr=32000)
wav = librosa.util.normalize(wav)
test_list=list()
model.eval()
classes_list = np.array(blist)[:-1]
for i in range(0,len(wav),32000*5):
    temp = wav[i:i+32000*5]
    temp = torch.from_numpy(temp[np.newaxis,:]).to(device)
    mel_spec = feature_extractor(temp)
    mel_spec = torch.from_numpy(mono_to_color(mel_spec.cpu().detach().numpy())).to(device)
    
    #print(mel_spec)
    output = torch.squeeze(F.softmax(model(mel_spec.to(torch.float32))))
    np_output = output.cpu().detach().numpy()
    print(classes_list[np_output>0.3])
    #detected_classes = 


In [ ]:
for i in dataset:
    print(classes_list[i[1]>=1])

In [ ]:
from sklearn.metrics import f1_score

a = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
b = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


print(len(a))
print(f1_score(a,b))